<a href="https://colab.research.google.com/github/StevenMorlier/Colab-LLM-Connect/blob/main/ollama_ngrok_roo_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# running Ollama on colab to test LLMs on Roo-Code Extention for VScode


## THREE-STEP PROCESS
━━━━━━━━━━━━━━━━━━━━━

### 01 │ SET UP NGROK

▢ Add `NGROK_TOKEN` to Colab secrets panel (◆)
  
  ▹ Obtain token: ngrok.com (1 free tunnel available)

━━━━━━━━━━━━━━━━━━━━━

### 02 │ EXECUTE NOTEBOOK

▢ Select "Runtime" → "Run all"

  ▹ Wait until completion signal

━━━━━━━━━━━━━━━━━━━━━

### 03 │ CONNECT TO CLINE

▢ Copy the generated ngrok URL
  
▢ Configure Roo-code settings with URL in the ollama API (remove the last "/")

  ▹ you're good to go 🚀

━━━━━━━━━━━━━━━━━━━━━


│ MODEL CONFIGURATION                         │

Modify first cell to change model:

MODEL_NAME = "llama3.3:latest"  # always specify :latest or size of the model (:8b or :14b...)


Thanks to Matthieu DELARO for is good advices,
Gholamreza Dar & maryasov for for their works (links just below)

https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab

https://github.com/maryasov/ollama-models-instruct-for-cline.git



---



#START HERE
but if you want to change the model, do it below

In [ ]:
MODEL_NAME = "maryasov/qwen2.5-coder-cline:7b-instruct-q8_0" # <--- Works well with Roo-code and T4 high RAM
#MODEL_NAME = "qwq:32b" # <-- Slow, should find an optimised version for Roo...
#MODEL_NAME = "tom_himanen/deepseek-r1-roo-cline-tools:32b" # <--- This one is too slow on T4
#see differents models on https://github.com/maryasov/ollama-models-instruct-for-cline.git

## colab GPU basic setting for Ollama

In [ ]:
!apt-get install -y lshw
!apt-get install -y pciutils
!nvcc --version
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

  from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('❌ Not using a high-RAM runtime')
else:
  print('✅ You are using a high-RAM runtime!')

In [ ]:
# Installing Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# needed for tunneling via ngrok
%env OLLAMA_HOST=0.0.0.0

#Setting the size of the context
#%env OLLAMA_CONTEXT_LENGTH=32768

# Running Ollama and serving on localhost:11434
!nohup ollama serve &
!ollama pull {MODEL_NAME}

## test the model

In [ ]:
# Test the API using curl locally with the MODEL_WITH_TEMPLATE variable
import json

# Préparation des données avec la variable MODEL_WITH_TEMPLATE
data = {
    "model": MODEL_NAME,
    "prompt": "Question: Who was the first president of the United States? \n Only answer using a few words. maybe just a name Answer: "
}

# Construction de la commande curl avec les données JSON
!curl http://localhost:11434/api/generate -d '{json.dumps(data)}'

## Expose the API publicly using ngrok

In [ ]:
!pip install -qU pyngrok

In [ ]:
from google.colab import userdata
from pyngrok import ngrok, conf

# get NGROK_TOKEN from colab secrets
ngrok_token = userdata.get('NGROK_TOKEN')
if not ngrok_token:
    raise ValueError("NGROK_TOKEN secret not found. Please add it to Colab secrets.")

# Set the ngrok auth token using Python
conf.get_default().auth_token = ngrok_token
ngrok.set_auth_token(ngrok_token)

# Expose Ollama server via ngrok on port 11434
public_url = ngrok.connect("http://localhost:11434")
print(f"Ollama server public URL for CLINE: {public_url.public_url}")
print(f"Model: {MODEL_NAME} (do NOT add / at the end in cline url)")
print(f"commande pour utiliser sur ollama en local")
print(f"export OLLAMA_HOST={public_url.public_url}/")

In [ ]:
# BEEP, work done !
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')